<a href="https://colab.research.google.com/github/teteumac/Acoplamento-Quartico-Anomalo/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%matplotlib inline

In [3]:
from tensorflow import keras

class Model:
    def __init__(self, input_shape, learning_rate):
        self.input_shape = input_shape
        self.learning_rate = learning_rate

    def __call__(self, n_hidden, n_neurons, dropout):

        input_shape = self.input_shape
        learning_rate = self.learning_rate

        print( "Building model with:" )
        print( "Input shape: {}".format(input_shape) )
        print( "Learning rate: {}".format(learning_rate) )
        print( "Number of hidden layers: {}".format(n_hidden) )
        print( "Number of neurons per layer: {}".format(n_neurons) )
        print( "Dropout rate: {}".format(dropout) )

        model = keras.models.Sequential()
        model.add( keras.layers.InputLayer(input_shape=input_shape) )
        for layer in range(n_hidden):
            if dropout > 0.:
                model.add( keras.layers.Dropout(rate=dropout) )
            model.add( keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") )
        if dropout > 0.:
            model.add( keras.layers.Dropout(rate=dropout) )
        model.add( keras.layers.Dense(1, activation="sigmoid") )

        #optimizer = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
        metrics = [
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall") ]
        optimizer = keras.optimizers.Nadam(lr=learning_rate)
        model.compile( loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)

        return model

def build_model(input_shape, learning_rate=5e-4, n_hidden=1, n_neurons=50, dropout=0.20 ):
    build_fn_ = Model( input_shape=input_shape, learning_rate=learning_rate )
    return build_fn_( n_hidden, n_neurons, dropout )

In [4]:
!python3 -m pip install coffea mplhep
!pip install matplotlib==3.1.3

     |████████████████████████████████| 163kB 11.7MB/s 
     |████████████████████████████████| 122kB 19.9MB/s 
     |████████████████████████████████| 215kB 23.1MB/s 
     |████████████████████████████████| 1.8MB 21.7MB/s 
     |████████████████████████████████| 8.1MB 36.7MB/s 
     |████████████████████████████████| 5.4MB 39.2MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 1.9MB 35.8MB/s 
ERROR: mplhep 0.3.6 has requirement matplotlib>=3.4, but you'll have matplotlib 3.2.2 which is incompatible.
     |████████████████████████████████| 13.1MB 10.2MB/s 
ERROR: mplhep 0.3.6 has requirement matplotlib>=3.4, but you'll have matplotlib 3.1.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [5]:
import os
import numpy as np
import pandas as pd
import h5py
from joblib import dump, load

import sklearn
import tensorflow as tf
from tensorflow import keras

print ( "sklearn: {}".format(sklearn.__version__) )
print ( "tensorflow: {}".format(tf.__version__) )

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import mplhep as hep
from pandas import set_option
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import h5py
from keras.models import load_model
from sklearn.metrics import auc,make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score

import argparse

from joblib import dump

sklearn: 0.22.2.post1
tensorflow: 2.4.1


In [6]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/'
plt.style.use(hep.style.ROOT)

def open_file_MC( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi       
        return dataframe

def open_file_DD( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]        
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt',
       'jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt',
       'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'MultiRP1', 'MultiRP2',
       'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi       
        return dataframe

def open_file_Data( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]        
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2','Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi        
        return dataframe

#select_columns = ['Mww', 'Pt_W_lep', 'jetAK8_pt','jetAK8_eta', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2','Mx', 'Mww/Mx']
select_columns = ['Mww', 'Pt_W_lep', 'Acoplanaridade_jatos_MET', 'Acoplanaridade_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'xi1', 'xi2', 'Mx', 'Yx', 'Mww/Mx']


SM =       open_file_MC( PATH + 'DataSet_SM_multiRP.h5')
ANOMALO1 = open_file_MC( PATH + 'DataSet_ANOMALO1_multiRP.h5')
ANOMALO2 = open_file_MC( PATH + 'DataSet_ANOMALO2_multiRP.h5')
ANOMALO3 = open_file_MC( PATH + 'DataSet_ANOMALO3_multiRP.h5')
ANOMALO4 = open_file_MC( PATH + 'DataSet_ANOMALO4_multiRP.h5')
ANOMALO5 = open_file_MC( PATH + 'DataSet_ANOMALO5_multiRP.h5')
ANOMALO6 = open_file_MC( PATH + 'DataSet_ANOMALO6_multiRP.h5')
ANOMALO7 = open_file_MC( PATH + 'DataSet_ANOMALO7_multiRP.h5')
ANOMALO8 = open_file_MC( PATH + 'DataSet_ANOMALO8_multiRP.h5')

label_signal1  = pd.DataFrame( [1]*len( ANOMALO1 ) )
label_signal2  = pd.DataFrame( [1]*len( ANOMALO2 ) )
label_signal3  = pd.DataFrame( [1]*len( ANOMALO3 ) )
label_signal4  = pd.DataFrame( [1]*len( ANOMALO4 ) )
label_signal5  = pd.DataFrame( [1]*len( ANOMALO5 ) )
label_signal6  = pd.DataFrame( [1]*len( ANOMALO6 ) )
label_signal7  = pd.DataFrame( [1]*len( ANOMALO7 ) )
label_signal8  = pd.DataFrame( [1]*len( ANOMALO8 ) )
label_signalSM = pd.DataFrame( [1]*len( SM ) )

SM = pd.concat( [ SM, label_signalSM ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Standard Model --> ', SM.shape)

ANOMALO1 = pd.concat( [ ANOMALO1, label_signal1 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 1 --> ', ANOMALO1.shape)

ANOMALO2 = pd.concat( [ ANOMALO2, label_signal2 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 2 --> ', ANOMALO2.shape)

ANOMALO3 = pd.concat( [ ANOMALO3, label_signal3 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 3 --> ', ANOMALO3.shape)

ANOMALO4 = pd.concat( [ ANOMALO4, label_signal4 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 4 --> ', ANOMALO4.shape)

ANOMALO5 = pd.concat( [ ANOMALO5, label_signal5 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 5 --> ', ANOMALO5.shape)

ANOMALO6 = pd.concat( [ ANOMALO6, label_signal6 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 6 --> ', ANOMALO6.shape)

ANOMALO7 = pd.concat( [ ANOMALO7, label_signal7 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 7 --> ', ANOMALO7.shape)

ANOMALO8 = pd.concat( [ ANOMALO8, label_signal8 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 8 --> ', ANOMALO8.shape)

data_set_back_multirp = open_file_DD( PATH + 'DataDriven_Background_multiRP.h5' )

label_back = pd.DataFrame( [0]*len( data_set_back_multirp ) )
data_set_back_multirp = pd.concat( [ data_set_back_multirp, label_back ], axis = 1 ).rename(columns={0: 'label'})

Dataset_Signal_Back1  = pd.concat( [ ANOMALO1 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back2  = pd.concat( [ ANOMALO2 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back3  = pd.concat( [ ANOMALO3 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back4  = pd.concat( [ ANOMALO4 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back5  = pd.concat( [ ANOMALO5 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back6  = pd.concat( [ ANOMALO6 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back7  = pd.concat( [ ANOMALO7 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back8  = pd.concat( [ ANOMALO8 , data_set_back_multirp, SM  ], axis = 0, sort = False )

data_set_dados_multirp = open_file_Data( PATH + 'DataSet_dados_multiRP.h5' )
data_set_dados_multirp = data_set_dados_multirp[select_columns]

from sklearn.model_selection import train_test_split

test_size = 0.35
DataSet_Train1_, DataSet_Test1_ = train_test_split( Dataset_Signal_Back1, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back1.label )
DataSet_Train2_, DataSet_Test2_ = train_test_split( Dataset_Signal_Back2, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back2.label )
DataSet_Train3_, DataSet_Test3_ = train_test_split( Dataset_Signal_Back3, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back3.label )
DataSet_Train4_, DataSet_Test4_ = train_test_split( Dataset_Signal_Back4, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back4.label )
DataSet_Train5_, DataSet_Test5_ = train_test_split( Dataset_Signal_Back5, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back5.label )
DataSet_Train6_, DataSet_Test6_ = train_test_split( Dataset_Signal_Back6, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back6.label )
DataSet_Train7_, DataSet_Test7_ = train_test_split( Dataset_Signal_Back7, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back7.label )
DataSet_Train8_, DataSet_Test8_ = train_test_split( Dataset_Signal_Back8, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back8.label )


DataSet_Train8 = DataSet_Train8_[select_columns] 
DataSet_Test8 = DataSet_Test8_[select_columns] 

DataSet_Train7 = DataSet_Train7_[select_columns] 
DataSet_Test7 = DataSet_Test7_[select_columns] 

DataSet_Train6 = DataSet_Train6_[select_columns] 
DataSet_Test6 = DataSet_Test6_[select_columns] 

DataSet_Train5 = DataSet_Train5_[select_columns] 
DataSet_Test5 = DataSet_Test5_[select_columns] 

DataSet_Train1 = DataSet_Train1_[select_columns] 
DataSet_Test1 = DataSet_Test1_[select_columns] 

DataSet_Train2 = DataSet_Train2_[select_columns] 
DataSet_Test2 = DataSet_Test2_[select_columns] 

DataSet_Train3 = DataSet_Train3_[select_columns] 
DataSet_Test3 = DataSet_Test3_[select_columns] 

DataSet_Train4 = DataSet_Train4_[select_columns] 
DataSet_Test4 = DataSet_Test4_[select_columns] 

y_train1 = DataSet_Train1_['label']
y_test1  = DataSet_Test1_['label']

y_train2 = DataSet_Train2_['label']
y_test2  = DataSet_Test2_['label']

y_train3 = DataSet_Train3_['label']
y_test3  = DataSet_Test3_['label']

y_train4 = DataSet_Train4_['label']
y_test4  = DataSet_Test4_['label']

y_train5 = DataSet_Train5_['label']
y_test5  = DataSet_Test5_['label']

y_train6 = DataSet_Train6_['label']
y_test6  = DataSet_Test6_['label']

y_train7 = DataSet_Train7_['label']
y_test7  = DataSet_Test7_['label']

y_train8 = DataSet_Train8_['label']
y_test8  = DataSet_Test8_['label']

print('--- Weight Anomalo 8 --- \n')
DataSet_Test8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==1]['weight'] 
DataSet_Test8_weight_backgr =DataSet_Test8_[DataSet_Test8_['label']==0]['weight'] 
DataSet_TestSM8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==2]['weight'] 

DataSet_Train8_weight_signal = DataSet_Train8_[DataSet_Train8_['label']==1]['weight']
DataSet_Train8_weight_backgr = DataSet_Train8_[DataSet_Train8_['label']==0]['weight'] 

DataSet_Test8_weight =  DataSet_Test8_['weight']

print( 'Shape Test Signal', DataSet_Test8_weight_signal.shape)
print( 'Shape Test Background', DataSet_Test8_weight_backgr.shape, '\n')

print('--- Weight Anomalo 7 --- \n')

DataSet_Test7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==1]['weight'] 
DataSet_Test7_weight_backgr =  DataSet_Test7_[DataSet_Test7_['label']==0]['weight'] 
DataSet_TestSM7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==2]['weight'] 

DataSet_Train7_weight_signal =  DataSet_Train7_[DataSet_Train7_['label']==1]['weight']
DataSet_Train7_weight_backgr =  DataSet_Train7_[DataSet_Train7_['label']==0]['weight'] 

DataSet_Test7_weight =  DataSet_Test7_['weight']

print('--- Weight Anomalo 6 --- \n')

DataSet_Test6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==1]['weight'] 
DataSet_Test6_weight_backgr =  DataSet_Test6_[DataSet_Test6_['label']==0]['weight'] 
DataSet_TestSM6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==2]['weight'] 

DataSet_Train6_weight_signal =  DataSet_Train6_[DataSet_Train6_['label']==1]['weight']
DataSet_Train6_weight_backgr =  DataSet_Train6_[DataSet_Train6_['label']==0]['weight'] 

DataSet_Test6_weight =  DataSet_Test6_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 5 --- \n')

DataSet_Test5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==1]['weight'] 
DataSet_Test5_weight_backgr =  DataSet_Test5_[DataSet_Test5_['label']==0]['weight'] 
DataSet_TestSM5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==2]['weight'] 

DataSet_Train5_weight_signal =  DataSet_Train5_[DataSet_Train5_['label']==1]['weight']
DataSet_Train5_weight_backgr =  DataSet_Train5_[DataSet_Train5_['label']==0]['weight'] 

DataSet_Test5_weight =  DataSet_Test5_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 1 --- \n')

DataSet_Test1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==1]['weight'] 
DataSet_Test1_weight_backgr =  DataSet_Test1_[DataSet_Test1_['label']==0]['weight'] 
DataSet_TestSM1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==2]['weight'] 

DataSet_Train1_weight_signal =  DataSet_Train1_[DataSet_Train1_['label']==1]['weight']
DataSet_Train1_weight_backgr = DataSet_Train1_[DataSet_Train1_['label']==0]['weight'] 

DataSet_Test1_weight =  DataSet_Test1_['weight']

print( 'Shape Test Signal', DataSet_Test1_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test1_weight_backgr.shape, '\n')

print('--- Weight Anomalo 2 --- \n')

DataSet_Test2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==1]['weight'] 
DataSet_Test2_weight_backgr =  DataSet_Test2_[DataSet_Test2_['label']==0]['weight'] 
DataSet_TestSM2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==2]['weight'] 

DataSet_Train2_weight_signal =  DataSet_Train2_[DataSet_Train2_['label']==1]['weight']
DataSet_Train2_weight_backgr = DataSet_Train2_[DataSet_Train2_['label']==0]['weight'] 

DataSet_Test2_weight =  DataSet_Test2_['weight']

print( 'Shape Test Signal', DataSet_Test2_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test2_weight_backgr.shape, '\n')

print('--- Weight Anomalo 3 --- \n')

DataSet_Test3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==1]['weight'] 
DataSet_Test3_weight_backgr =  DataSet_Test3_[DataSet_Test3_['label']==0]['weight'] 
DataSet_TestSM3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==2]['weight'] 

DataSet_Train3_weight_signal =  DataSet_Train3_[DataSet_Train3_['label']==1]['weight']
DataSet_Train3_weight_backgr = DataSet_Train3_[DataSet_Train3_['label']==0]['weight'] 

DataSet_Test3_weight =  DataSet_Test3_['weight']

print( 'Shape Test Signal', DataSet_Test3_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test3_weight_backgr.shape, '\n')

print('--- Weight Anomalo 4 --- \n')

DataSet_Test4_weight_signal = DataSet_Test4_[DataSet_Test4_['label']==1]['weight'] 
DataSet_Test4_weight_backgr = DataSet_Test4_[DataSet_Test4_['label']==0]['weight'] 
DataSet_TestSM4_weight_signal =  DataSet_Test4_[DataSet_Test4_['label']==2]['weight'] 

DataSet_Train4_weight_signal = DataSet_Train4_[DataSet_Train4_['label']==1]['weight']
DataSet_Train4_weight_backgr = DataSet_Train4_[DataSet_Train4_['label']==0]['weight'] 

DataSet_Test4_weight =  DataSet_Test4_['weight']

print( 'Shape Test Signal' , DataSet_Test4_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test4_weight_backgr.shape)

Shape for Standard Model -->  (112, 24)
Shape for Anomalo 1 -->  (1544, 24)
Shape for Anomalo 2 -->  (174, 24)
Shape for Anomalo 3 -->  (434, 24)
Shape for Anomalo 4 -->  (742, 24)
Shape for Anomalo 5 -->  (169, 24)
Shape for Anomalo 6 -->  (299, 24)
Shape for Anomalo 7 -->  (672, 24)
Shape for Anomalo 8 -->  (1627, 24)
--- Weight Anomalo 8 --- 

Shape Test Signal (609,)
Shape Test Background (146071,) 

--- Weight Anomalo 7 --- 

--- Weight Anomalo 6 --- 

Shape Test Signal (274,)
Shape Test Background (146072,) 

--- Weight Anomalo 5 --- 

Shape Test Signal (274,)
Shape Test Background (146072,) 

--- Weight Anomalo 1 --- 

Shape Test Signal (580,)
Shape Test Background (146071,) 

--- Weight Anomalo 2 --- 

Shape Test Signal (100,)
Shape Test Background (146071,) 

--- Weight Anomalo 3 --- 

Shape Test Signal (191,)
Shape Test Background (146071,) 

--- Weight Anomalo 4 --- 

Shape Test Signal (299,)
Shape Test Background (146071,)


In [7]:
X_train1, X_valid1, y_train1, y_valid1 = train_test_split( DataSet_Train1, y_train1, test_size=0.20, shuffle=True, random_state=42 )
X_train2, X_valid2, y_train2, y_valid2 = train_test_split( DataSet_Train2, y_train2, test_size=0.20, shuffle=True, random_state=42 )
X_train3, X_valid3, y_train3, y_valid3 = train_test_split( DataSet_Train3, y_train3, test_size=0.20, shuffle=True, random_state=42 )
X_train4, X_valid4, y_train4, y_valid4 = train_test_split( DataSet_Train4, y_train4, test_size=0.20, shuffle=True, random_state=42 )
X_train5, X_valid5, y_train5, y_valid5 = train_test_split( DataSet_Train5, y_train5, test_size=0.20, shuffle=True, random_state=42 )
X_train6, X_valid6, y_train6, y_valid6 = train_test_split( DataSet_Train6, y_train6, test_size=0.20, shuffle=True, random_state=42 )
X_train7, X_valid7, y_train7, y_valid7 = train_test_split( DataSet_Train7, y_train7, test_size=0.20, shuffle=True, random_state=42 )
X_train8, X_valid8, y_train8, y_valid8 = train_test_split( DataSet_Train8, y_train8, test_size=0.20, shuffle=True, random_state=42 )

scaler = StandardScaler()

X_train_scaled1 = scaler.fit_transform( X_train1 )
X_valid_scaled1 = scaler.transform( X_valid1)
X_test_scaled1 = scaler.transform( DataSet_Test1 )

X_train_scaled2 = scaler.fit_transform( X_train2 )
X_valid_scaled2 = scaler.transform( X_valid2 )
X_test_scaled2 = scaler.transform( DataSet_Test2 )

X_train_scaled3 = scaler.fit_transform( X_train3 )
X_valid_scaled3 = scaler.transform( X_valid3 )
X_test_scaled3 = scaler.transform( DataSet_Test3 )

X_train_scaled4 = scaler.fit_transform( X_train4 )
X_valid_scaled4 = scaler.transform( X_valid4 )
X_test_scaled4 = scaler.transform( DataSet_Test4 )

X_train_scaled5 = scaler.fit_transform( X_train5 )
X_valid_scaled5 = scaler.transform( X_valid5 )
X_test_scaled5 = scaler.transform( DataSet_Test5 )

X_train_scaled6 = scaler.fit_transform( X_train6 )
X_valid_scaled6 = scaler.transform( X_valid6 )
X_test_scaled6 = scaler.transform( DataSet_Test6 )

X_train_scaled7 = scaler.fit_transform( X_train7 )
X_valid_scaled7 = scaler.transform( X_valid7 )
X_test_scaled7 = scaler.transform( DataSet_Test7 )

X_train_scaled8 = scaler.fit_transform( X_train8 )
X_valid_scaled8 = scaler.transform( X_valid8 )
X_test_scaled8 = scaler.transform( DataSet_Test8 )

grid_search = None
run_tables = False
train_model = True
run_grid_search = True
save_model = True
n_iter_search_ = 100

learning_rate = 5e-4
epochs_grid_search = 20

In [8]:
learning_rate = 5e-4
epochs_grid_search = 20

print ( scaler )

'''if train_model and save_model:
    import time
    id_ = time.strftime("%Y_%m_%d-%H_%M_%S")
    fileName_ = "standard_scaler_{}_{}.joblib".format( label_anomalo, id_ )
    print ( "Saving scaler to {}".format( fileName_ ) )
    dump( scaler, fileName_ )
'''
print('X_train_scaled --> ', X_train_scaled1 )


def get_run_logdir(log_dir):
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(log_dir, run_id)

def callbacks(patience=10, log_dir=""):
    callbacks_ = []
    # Early stopping
    if patience > 0:
        early_stopping_cb_ = keras.callbacks.EarlyStopping( patience=patience, restore_best_weights=True )
        callbacks_.append( early_stopping_cb_ )

    # TensorBoard
    if log_dir:
        run_logdir = get_run_logdir(log_dir)
        print ( "Log dir: {}".format(run_logdir) )
        tensorboard_cb_ = keras.callbacks.TensorBoard( run_logdir )
        callbacks_.append( tensorboard_cb_ )

    return callbacks_

grid_search = None


StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_scaled -->  [[-1.78258180e-01 -5.18161560e-03 -7.58058168e-01 ... -1.40345129e+00
   1.30959803e+00  4.87261128e-02]
 [-2.56317444e-03 -3.44127926e-03 -4.87964280e-01 ... -9.61138782e-02
   2.65879059e-01 -1.81266971e-02]
 [ 4.71130822e-01 -8.96809555e-03 -5.78768826e-01 ...  2.59822319e-01
   2.98671486e-01  3.14498644e-01]
 ...
 [-1.46552280e-01 -4.56944309e-03 -4.63490524e-01 ...  1.39366217e-01
  -1.28141105e-01 -1.72401361e-01]
 [ 2.69940925e-01 -1.03431065e-02 -2.50121758e-01 ... -7.60007198e-01
  -9.26890729e-01  3.68948169e-01]
 [-2.40142570e-01 -1.35441609e-03 -6.32688708e-01 ...  8.43180493e-01
   1.09897599e+00 -3.16514719e-01]]


In [9]:
'''if train_model and run_grid_search:
    import time
    print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
    time_s_ = time.time()

    from sklearn.model_selection import RandomizedSearchCV
    #from sklearn.model_selection import GridSearchCV

    build_fn_ = Model( input_shape=X_train_scaled1.shape[1:], learning_rate=learning_rate )
    keras_clf = keras.wrappers.scikit_learn.KerasClassifier( build_fn_ )

#     param_grid = [
#         { "n_hidden": np.arange(1,3),
#           "n_neurons": [20,50] }
#         ]

    param_distribs = {
        "n_hidden": np.arange(2,6),
        "n_neurons": 2 ** np.arange(4,8),
        "dropout":  0.1 * np.arange(2,6),
        "batch_size": 2 ** np.arange(5,8)
        }

    #grid_search = GridSearchCV( keras_clf, param_grid, cv=3, scoring='f1', refit=False )

    grid_search = RandomizedSearchCV(
        keras_clf,
        param_distribs,
        n_iter=n_iter_search_, cv=3, verbose=20, n_jobs=-1, scoring='f1', refit=False, random_state=42
        )

    callbacks_ = callbacks(patience=5)
    print ( callbacks_ )
    grid_search.fit( X_train_scaled1, y_train8, epochs=epochs_grid_search, validation_data=(X_valid_scaled1, y_valid8), callbacks=callbacks_ )

    print ( grid_search.best_params_ )
    print ( grid_search.best_score_ )
    print ( grid_search.cv_results_ )

    time_e_ = time.time()
    print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

model_final = None'''

'if train_model and run_grid_search:\n    import time\n    print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )\n    time_s_ = time.time()\n\n    from sklearn.model_selection import RandomizedSearchCV\n    #from sklearn.model_selection import GridSearchCV\n\n    build_fn_ = Model( input_shape=X_train_scaled1.shape[1:], learning_rate=learning_rate )\n    keras_clf = keras.wrappers.scikit_learn.KerasClassifier( build_fn_ )\n\n#     param_grid = [\n#         { "n_hidden": np.arange(1,3),\n#           "n_neurons": [20,50] }\n#         ]\n\n    param_distribs = {\n        "n_hidden": np.arange(2,6),\n        "n_neurons": 2 ** np.arange(4,8),\n        "dropout":  0.1 * np.arange(2,6),\n        "batch_size": 2 ** np.arange(5,8)\n        }\n\n    #grid_search = GridSearchCV( keras_clf, param_grid, cv=3, scoring=\'f1\', refit=False )\n\n    grid_search = RandomizedSearchCV(\n        keras_clf,\n        param_distribs,\n        n_iter=n_iter_search_, cv=3, verbose=20, n_jobs=-1, scorin

In [10]:
def traning_keras( X_train_scaled, y_train, X_valid_scaled, y_valid, X_test_scaled, y_test, DataSet_Test_weight_backgr, DataSet_Test_weight_signal, label_anomalo ):
    if train_model and run_grid_search:
        import time
        print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
        time_s_ = time.time()

        from sklearn.model_selection import RandomizedSearchCV
        #from sklearn.model_selection import GridSearchCV

        build_fn_ = Model( input_shape=X_train_scaled.shape[1:], learning_rate=learning_rate )
        keras_clf = keras.wrappers.scikit_learn.KerasClassifier( build_fn_ )

    #     param_grid = [
    #         { "n_hidden": np.arange(1,3),
    #           "n_neurons": [20,50] }
    #         ]

        param_distribs = {
            "n_hidden": np.arange(2,10),
            "n_neurons": 2 ** np.arange(5,9),
            "dropout":  0.1 * np.arange(3,8),
            "batch_size": 2 ** np.arange(5,8)
            }

        #grid_search = GridSearchCV( keras_clf, param_grid, cv=3, scoring='f1', refit=False )

        grid_search = RandomizedSearchCV(
            keras_clf,
            param_distribs,
            n_iter=n_iter_search_, cv=2, verbose=20, n_jobs=-1, scoring='f1', refit=False, random_state=42
            )

        callbacks_ = callbacks(patience=5)
        print ( callbacks_ )
        grid_search.fit( X_train_scaled, y_train, epochs=epochs_grid_search, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks_ )

        print ( grid_search.best_params_ )
        print ( grid_search.best_score_ )
        print ( grid_search.cv_results_ )

        time_e_ = time.time()
        print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

    model_final = None
    '''
    if train_model:
        params = {'n_hidden': 1, 'n_neurons': 50, 'dropout': 0.20}
        batch_size = 32
        if run_grid_search:
            params = grid_search.best_params_.copy()
            batch_size = params[ 'batch_size' ]
            params.pop( 'batch_size' )
        print ( params, "batch_size: {}".format( batch_size ) )

        model_final = build_model(input_shape=X_train_scaled.shape[1:], learning_rate=learning_rate, **params )
        model_final.summary()
        #log_dir="keras_logs"
        #callbacks_ = callbacks(patience=5, log_dir=log_dir)
        callbacks_ = callbacks( patience=5 )
        print ( callbacks_ )
        model_final.fit( X_train_scaled, y_train, epochs=200, batch_size=batch_size, validation_data=(X_valid_scaled, y_valid), callbacks=None, verbose = 1 )
    else:
        model_final = keras.models.load_model( "/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/keras_model_"+label_anomalo+".h5" )
    '''

    model_final = build_model(input_shape=X_train_scaled.shape[1:], learning_rate=learning_rate, **grid_search.best_params_ )
    model_final.summary()
    callbacks_ = callbacks( patience=5 )
    print ( 'callbacks--> ', callbacks_ )
    model_final.fit( X_train_scaled, y_train, epochs=200, batch_size=batch_size, validation_data=(X_valid_scaled, y_valid), callbacks=None, verbose = 1 )

    # Evaluate on training data (without dropout)

    model_final.evaluate( X_train_scaled, y_train )

    model_final.evaluate( X_valid_scaled, y_valid )

    # Evaluate on test data

    model_final.evaluate( X_test_scaled, y_test )    

    model_final.save("/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/keras_model_"+label_anomalo+".h5") 

    y_test_proba = model_final.predict( X_test_scaled )
    print( 'y_test_proba --> ', y_test_proba )

    y_train_proba = model_final.predict( X_train_scaled )
    print( 'y_train_proba --> ', y_train_proba )

    prec_test, rec_test, thresh_test = precision_recall_curve(y_test, y_test_proba)
    bidx_test = np.argmax(prec_test * rec_test)
    prob_cut_test = thresh_test[bidx_test]

    prec_train, rec_train, thresh_train = precision_recall_curve(y_train, y_train_proba)
    bidx_train = np.argmax(prec_train * rec_train)
    prob_cut_train = thresh_train[bidx_train]

    print ( "Prob. cut test: {}".format( prob_cut_test ) )

    y_test_pred = ( y_test_proba >= prob_cut_test ).astype( "int32" )
    print ( y_test_pred )

    print ( "Prob. cut train: {}".format( prob_cut_train ) )

    y_train_pred = ( y_train_proba >= prob_cut_train ).astype( "int32" )
    print ( y_train_pred )

    print( "F1_score in TEST sample  : {:2.2f}%".format(100 * f1_score(y_test, y_test_pred)))
    print( "F1_score in TRAIN sample  : {:2.2f}%".format(100 * f1_score(y_train, y_train_pred)))

    #print ( f1_score( y_test1[ y_test1 == 1 ], y_test_pred[ y_test1 == 1 ] ) )
    #print ( f1_score( y_test1[ y_test1 == 0 ], y_test_pred[ y_test1 == 0 ] ) )    

    plt.hist( y_test_proba[y_test == 0], bins = 25, range = (0,1) , weights=DataSet_Test_weight_backgr )
    plt.hist( y_test_proba[y_test == 1], bins = 25, range = (0,1) , histtype='step', weights=DataSet_Test_weight_signal )
    plt.yscale('log')    

In [ ]:
traning_keras( X_train_scaled1, y_train1, X_valid_scaled1, y_valid1, X_test_scaled1, y_test1, DataSet_Test1_weight_backgr, DataSet_Test1_weight_signal, "anomalo1_SM_como_Sinal" )

2021/05/17 11:42:42
Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 28.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 29.0min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 29

In [ ]:
traning_keras( X_train_scaled2, y_train2, X_valid_scaled2, y_valid2, X_test_scaled2, y_test2, DataSet_Test2_weight_backgr, DataSet_Test2_weight_signal, "anomalo2_SM_como_sinal" )

In [ ]:
traning_keras( X_train_scaled3, y_train3, X_valid_scaled3, y_valid3, X_test_scaled3, y_test3, DataSet_Test3_weight_backgr, DataSet_Test3_weight_signal, "anomalo3_SM_como_sinal" )

In [ ]:
traning_keras( X_train_scaled4, y_train4, X_valid_scaled4, y_valid4, X_test_scaled4, y_test4, DataSet_Test4_weight_backgr, DataSet_Test4_weight_signal, "anomalo4_SM_como_sinal" )

In [ ]:
if train_model:
    params = {'n_hidden': 1, 'n_neurons': 50, 'dropout': 0.20}
    batch_size = 32
    if run_grid_search:
        params = grid_search.best_params_.copy()
        batch_size = params[ 'batch_size' ]
        params.pop( 'batch_size' )
    print ( params, "batch_size: {}".format( batch_size ) )

    model_final = build_model(input_shape=X_train_scaled1.shape[1:], learning_rate=learning_rate, **params )
    model_final.summary()
    #log_dir="keras_logs"
    #callbacks_ = callbacks(patience=5, log_dir=log_dir)
    callbacks_ = callbacks( patience=5 )
    print ( callbacks_ )
    model_final.fit( X_train_scaled1, y_train8, epochs=100, batch_size=batch_size, validation_data=(X_valid_scaled1, y_valid8), callbacks=callbacks_ )
else:
    model_final = keras.models.load_model( "/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/keras_model_anomalo8.h5" )

model_final.summary()
# Evaluate on training data (without dropout)

model_final.evaluate( X_train_scaled1, y_train8 )

model_final.evaluate( X_valid_scaled1, y_valid8 )

# Evaluate on test data

model_final.evaluate( X_test_scaled1, y_test8 )



In [ ]:
model_final.save("/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/keras_model_anomalo8.h5") 

In [ ]:
from sklearn.metrics import auc,make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score

In [ ]:
y_test_proba = model_final.predict( X_test_scaled1 )
print( 'y_test_proba --> ', y_test_proba )

y_train_proba = model_final.predict( X_train_scaled1 )
print( 'y_train_proba --> ', y_train_proba )

prec_test, rec_test, thresh_test = precision_recall_curve(y_test8, y_test_proba)
bidx_test = np.argmax(prec_test * rec_test)
prob_cut_test = thresh_test[bidx_test]

prec_train, rec_train, thresh_train = precision_recall_curve(y_train8, y_train_proba)
bidx_train = np.argmax(prec_train * rec_train)
prob_cut_train = thresh_train[bidx_train]

print ( "Prob. cut test: {}".format( prob_cut_test ) )

y_test_pred = ( y_test_proba >= prob_cut_test ).astype( "int32" )
print ( y_test_pred )

print ( "Prob. cut train: {}".format( prob_cut_train ) )

y_train_pred = ( y_train_proba >= prob_cut_train ).astype( "int32" )
print ( y_train_pred )

print( "F1_score in TEST sample  : {:2.2f}%".format(100 * f1_score(y_test8, y_test_pred)))
print( "F1_score in TRAIN sample  : {:2.2f}%".format(100 * f1_score(y_train8, y_train_pred)))

#print ( f1_score( y_test1[ y_test1 == 1 ], y_test_pred[ y_test1 == 1 ] ) )
#print ( f1_score( y_test1[ y_test1 == 0 ], y_test_pred[ y_test1 == 0 ] ) )

In [ ]:
plt.hist( y_test_proba[y_test8 == 0], bins = 25, range = (0,1) , weights=DataSet_Test8_weight_backgr )
plt.hist( y_test_proba[y_test8 == 1], bins = 25, range = (0,1) , histtype='step', weights=DataSet_Test8_weight_signal )
plt.yscale('log')